In [1]:
import os
from glob import glob
from pymystem3 import Mystem
import sqlite3
import pandas as pd
from pyspark.sql import SQLContext, SparkSession
from pyspark import pandas as ps

/home/al/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
m = Mystem()
PATH_DD = '/home/al/Projects_My/hakaton_ozon_org/'
#PATH_DD = '/home/gansiorag/MyParth/ProjectMy/hakaton_ozon_org/'

In [3]:
db_path = PATH_DD + 'dataset/ozon.db'
query = """SELECT name 
FROM sqlite_schema
WHERE type ='table' AND 
    name NOT LIKE 'sqlite_%';"""

conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute(query)
print(cur.fetchall())


[('products',)]


In [4]:
query = """SELECT name FROM PRAGMA_TABLE_INFO('products');"""
cur.execute(query)
for pp in cur.fetchall():
    print(pp[0])

search_nm
item_num
link
short_characteristic
full_desc
img_desc
img_path_desc
characteristic
components
conn_dev
img_path


In [5]:
query = """SELECT search_nm, item_num, link, short_characteristic,full_desc,
img_desc,
img_path_desc,
characteristic,
components,
conn_dev,
img_path 
FROM products;"""
a_pandas_df = pd.read_sql_query(query, conn)

In [6]:
a_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   search_nm             4398 non-null   object
 1   item_num              4398 non-null   object
 2   link                  4398 non-null   object
 3   short_characteristic  4398 non-null   object
 4   full_desc             4398 non-null   object
 5   img_desc              4398 non-null   object
 6   img_path_desc         4398 non-null   object
 7   characteristic        4398 non-null   object
 8   components            4398 non-null   object
 9   conn_dev              4398 non-null   object
 10  img_path              4398 non-null   object
dtypes: object(11)
memory usage: 378.1+ KB


In [7]:
spark = SparkSession.builder.appName('all_dataset').getOrCreate()
print('Spark info :')
spark


24/08/06 10:33:23 WARN Utils: Your hostname, al-Dell resolves to a loopback address: 127.0.1.1; using 192.168.0.106 instead (on interface wlp0s20f3)
24/08/06 10:33:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/06 10:33:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark info :


In [8]:
a_spark_df = ps.from_pandas(a_pandas_df)
df = a_spark_df.to_spark()

/home/al/.local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [9]:
df.show(5)

24/08/06 10:33:33 WARN TaskSetManager: Stage 0 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


+---------+----------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------+--------------------+
|search_nm|  item_num|                link|short_characteristic|           full_desc|img_desc|       img_path_desc|      characteristic|          components|conn_dev|            img_path|
+---------+----------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+--------+--------------------+
| смартфон|1409765841|https://www.ozon....|О товаре Перейти ...|Описание Смартфон...|        |dataset/смартфон/...|Характеристики До...|Смартфон, кабель ...|        |dataset/смартфон/...|
| смартфон|1624251355|https://www.ozon....|О товаре Перейти ...|Комплектация Комп...|        |                    |Характеристики До...|Комплект поставки...|        |dataset/смартфон/...|
| смартфон|1337138963|https://www.ozon....|О товаре Перейти 

In [10]:
example_row_full_desc = df.select(['full_desc', 'link']).collect()


24/08/06 10:33:39 WARN TaskSetManager: Stage 1 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
for rr in range(100, 110):
    print(example_row_full_desc[rr]['full_desc'], example_row_full_desc[rr]['link'])

Описание Операционная система - Операционная система: Android - Версия операционной системы: 14 Основные характеристики - Количество SIM-карт: 2 - Тип SIM-карты: 2 nano SIM - Режим работы нескольких SIM карт: попеременный - Модуль сотовой связи: 5G, 4G (LTE), 3G, 2G - Поддержка 5G: да - Поддержка NFC: есть - Поддержка eSIM: да Дисплей - Экран: 6.2" - Разрешение экрана: 2340x1080 - Яркость: 2600 кд/м2 - Тип дисплея: Dynamic AMOLED 2X - Количество цветов дисплея: 16 млн - Плотность пикселей: 416 PPI - Тип стекла: Corning Gorilla Glass Victus 2 - Функция Multi-Touch: да - Безрамочный дисплей: да - Частота обновления экрана: 120 Гц Процессор - Производитель процессора: Samsung - Модель процессора: Exynos 2400 - Частота процессора: 3200, 2х2900, 3х2600, 4х1950 МГц - Количество ядер процессора: 10 - Видеопроцессор: Samsung Xclipse 940 Память - Встроенная память: 128 Гб - Оперативная память: 8 Гб Основная камера - Количество основных камер: 3 - Разрешение основной камеры: 12 Мп, 10 Мп, 50 Мп 

In [13]:
type_obj = df.select('search_nm').distinct().collect()
for i in type_obj:
    print(i['search_nm'])

24/08/06 10:35:08 WARN TaskSetManager: Stage 2 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


смартфон
планшет
ноутбук
кроссовки
телевизор
туфли
стул
стол
платье
наушники
чехол для телефона


In [14]:
# Получение словаря dataframe для всех категорий
dict_txt_cat = {}
for ik in type_obj:
    d_serv = df.filter(df['search_nm'] == ik['search_nm'])
    dict_txt_cat[ik['search_nm']] = d_serv
    
list(dict_txt_cat.keys())[0]

'смартфон'

In [15]:
def func(value):
    for i in range(10):
        value = value.replace(str(i),'')
    value = value.replace('(-)', ' ')
    ddel = ['*', ',', '-', '/', '%', ';', ')', '(', '+', '.', '..', ':', '"',
 'c',
 'h',
 '«',
 '°',
  '–',
 '—',
 '•']
    for i in ddel:
        value = value.replace(i, ' ')
    return value

In [16]:
def get_array_text_with_category(all_text_list: list, m):
    """ Списка слов из списка текстов

    Args:
        all_text_list (list): Список текстов
        m (_type_): _description_
    """
    sum_arr = []
    for first_txt in all_text_list:
        words_list = func(first_txt.lower()).split()
        for word in words_list:
            # print(word)
            word = word.strip()
            if word:
                lemmas = m.lemmatize(word)
                sum_arr.append(lemmas[0])
    return sum_arr

# Получение списка слов в каждой категориии и создаие словаря категория:список слов

In [17]:
def get_array_cat_words(dict_txt_cat: dict):
    """ Список слов в каждой категории

    Args:
        dict_txt_cat (dict): Словарь категорий
    """
    dict_cat_words = {}
    for key in list(dict_txt_cat.keys()):
        print(key)
        dict_cat_words[key] = []
        ddf = dict_txt_cat[key]
        dict_cat_words[key]+= get_array_text_with_category([kk['short_characteristic'] for kk in ddf.select('short_characteristic').collect()], m)
        dict_cat_words[key]+= get_array_text_with_category([kk['full_desc'] for kk in ddf.select('full_desc').collect()], m)
        dict_cat_words[key]+= get_array_text_with_category([kk['characteristic'] for kk in ddf.select('characteristic').collect()], m)
        dict_cat_words[key]+= get_array_text_with_category([kk['components'] for kk in ddf.select('components').collect()], m)
        dict_cat_words[key] = set(dict_cat_words[key])
    return dict_cat_words


In [18]:
dict_cat_words = get_array_cat_words(dict_txt_cat)

смартфон


24/08/06 10:35:47 WARN TaskSetManager: Stage 5 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:35:48 WARN TaskSetManager: Stage 6 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:35:49 WARN TaskSetManager: Stage 7 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:35:51 WARN TaskSetManager: Stage 8 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


планшет


24/08/06 10:35:51 WARN TaskSetManager: Stage 9 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:35:52 WARN TaskSetManager: Stage 10 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:35:53 WARN TaskSetManager: Stage 11 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:35:55 WARN TaskSetManager: Stage 12 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


ноутбук


...

24/08/06 10:36:32 WARN TaskSetManager: Stage 37 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:33 WARN TaskSetManager: Stage 38 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:36 WARN TaskSetManager: Stage 39 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:40 WARN TaskSetManager: Stage 40 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


наушники


24/08/06 10:36:40 WARN TaskSetManager: Stage 41 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:41 WARN TaskSetManager: Stage 42 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:45 WARN TaskSetManager: Stage 43 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:49 WARN TaskSetManager: Stage 44 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


чехол для телефона


24/08/06 10:36:50 WARN TaskSetManager: Stage 45 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:51 WARN TaskSetManager: Stage 46 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:54 WARN TaskSetManager: Stage 47 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.
24/08/06 10:36:56 WARN TaskSetManager: Stage 48 contains a task of very large size (1677 KiB). The maximum recommended task size is 1000 KiB.


In [19]:
dict_cat_words['наушники']

{'неудобный',
 'сражение',
 'plantroni',
 'p',
 'отвлечение',
 'товарный',
 'вынимание',
 'автоматический',
 'наконец',
 'держать',
 'nb',
 'эффективный',
 'lg',
 'роль',
 'lo',
 'виртуальный',
 'прозрачность',
 'jснащены',
 'позолотить',
 'профессиональный',
 'ножевой',
 'уверенный',
 'руль',
 'ситуация',
 'воз',
 'три',
 'копия',
 'оранжевый',
 'сверхбыстрый',
 'потребность',
 'аудиокодек',
 'sven',
 'dj',
 'эффективность',
 'сеанс',
 'арсенал',
 'мера',
 'достигаться',
 'нагрузка',
 'унция',
 'метро',
 'задавать',
 'современный',
 'оголовье',
 'переносок',
 'басистый',
 'фон',
 'max',
 'эстетика',
 'упаковочный',
 'продукция',
 'выгодный',
 'максимум',
 'непринужденный',
 'передвигаться',
 'нож',
 'перемотка',
 'плотный',
 'raa',
 'производиль',
 'мода',
 'подвеска',
 'затылочный',
 'inpods',
 'бережный',
 'в',
 'игровой',
 'настроенный',
 'просьба',
 'совместимость',
 'зона',
 'поговорить',
 'слышимость',
 'сегодня',
 'хаки',
 'признанный',
 'iabbegru',
 'отклонение',
 'so',
 'goga

In [20]:
len(dict_cat_words['наушники'])

4399

In [21]:
list_dict_words = [dict_cat_words[ll] for ll in dict_cat_words]

In [22]:
len(list_dict_words)

11

In [23]:
set_com_words = set.intersection(*list_dict_words)

In [24]:
len(set_com_words)

340

In [25]:
set_com_words

{'a',
 'e',
 'f',
 'k',
 's',
 't',
 'а',
 'аксессуар',
 'активный',
 'артикул',
 'без',
 'белый',
 'благодаря',
 'более',
 'больше',
 'большой',
 'бояться',
 'бренд',
 'быстро',
 'быстрый',
 'быть',
 'в',
 'важный',
 'ваш',
 'верхний',
 'вес',
 'весь',
 'вид',
 'включать',
 'включая',
 'вместе',
 'внешний',
 'внимание',
 'внутренний',
 'во',
 'воздух',
 'возможность',
 'вокруг',
 'время',
 'все',
 'всегда',
 'всего',
 'вы',
 'выбирать',
 'выбор',
 'выглядеть',
 'выполнять',
 'высокий',
 'высококачественный',
 'высота',
 'гамма',
 'гарантийный',
 'гарантировать',
 'гарантия',
 'где',
 'глубокий',
 'год',
 'давать',
 'даже',
 'данный',
 'два',
 'движение',
 'делать',
 'день',
 'деталь',
 'дизайн',
 'длительный',
 'для',
 'до',
 'добавлять',
 'долгий',
 'долговечность',
 'дополнительный',
 'достаточно',
 'доступный',
 'друг',
 'другой',
 'если',
 'еще',
 'же',
 'жизнь',
 'за',
 'зависимость',
 'занимать',
 'защищать',
 'и',
 'игра',
 'идеально',
 'идеальный',
 'идти',
 'из',
 'изготавлив

In [32]:
set_big_com = set()
for i in range(len(list_dict_words) - 1):
    for j in range(i + 1, len(list_dict_words)):
        set_com_duble_words = set(list_dict_words[i]).intersection(set(list_dict_words[j]))
        set_big_com = set_big_com.union(set_com_duble_words)
print(len(set_big_com))
print(set_big_com)

5484
{'вещание', 'протирать', 'комбинация', 'прорезинивать', 'травма', 'держать', 'эффективный', 'olor', 'lg', 'роль', 'lo', 'пенополиуретан', 'уверенный', 'женственный', 'комплектность', 'ситуация', 'завязывание', 'практично', 'сверхбыстрый', 'временный', 'голливудский', 'эффективность', 'переворот', 'сеанс', 'sk', 'кормление', 'горный', 'арсенал', 'мера', 'нагрузка', 'star', 'задавать', 'современный', 'турция', 'переносок', 'вино', 'эстетика', 'max', 'продукция', 'выгодный', 'максимум', 'непринужденный', 'нож', 'передвигаться', 'перемотка', 'микроклимат', 'допы', 'бережный', 'игровой', 'совместимость', 'хаки', 'температурный', 'отклонение', 'всевозможный', 'отвлекаться', 'печатный', 'москва', 'олицетворение', 'доставлять', 'сообщение', 'нежелательный', 'свадьба', 'пышный', 'тпу', 'натирать', 'отображение', 'web', 'торжественный', 'мощь', 'производить', 'запатентовывать', 'презентация', 'дождаться', 'instreet', 'главный', 'бежевый', 'различать', 'магнитная', 'многоканальный', 'стройно

In [35]:
with open('/home/al/Projects_My/hakaton_ozon_org/dataset/stop_word_base.txt', 'w') as f:
    for word in set_big_com:
        f.write(word + '\n')


#### План
1. Распознавание категориальное <br>
2. Распознавание внутрикатегориальное <br>
     <br>
    - по каждой категории свой набор <br>
    - внутри каждой категории наборы по  <br>
        short_characteristic <br>
        full_desc <br>
        characteristic <br>
        components <br>
        с цифрами и без цифр <br> <br>
    <b>распознавание по 9 наборам слов - </b><br>
        общий <br>
        short_characteristic <br>
        full_desc <br>
        characteristic <br>
        components <br>
        short_characteristic <b>с цифрами </b><br>
        full_desc <b>с цифрами </b><br>
        characteristic <b>с цифрами </b><br>
        components <b>с цифрами </b><br>
         <br>

### Проба фрагментов

In [38]:
ddl = ddf.select('short_characteristic').collect()
list_text_one_category = [kk['short_characteristic'] for kk in ddl]
len(list_text_one_category)

24/08/02 09:02:13 WARN TaskSetManager: Stage 14 contains a task of very large size (1433 KiB). The maximum recommended task size is 1000 KiB.


140

In [39]:
list_words = get_array_text_with_category(list_text_one_category, m)

In [40]:
list_words

['о',
 'товар',
 'переходить',
 'к',
 'описание',
 'тип',
 'смартфон',
 'диагональ',
 'экран',
 'дюйм',
 'емкость',
 'аккумулятор',
 'мач',
 'процессор',
 'snapdragon',
 'ядро',
 'ггц',
 'основной',
 'материал',
 'корпус',
 'пластик',
 'стекло',
 'о',
 'товар',
 'переходить',
 'к',
 'характеристика',
 'тип',
 'смартфон',
 'диагональ',
 'экран',
 'дюйм',
 'емкость',
 'аккумулятор',
 'мач',
 'процессор',
 'snapdragon',
 'gen',
 'ядро',
 'ггц',
 'основной',
 'материал',
 'корпус',
 'металл',
 'стекло',
 'о',
 'товар',
 'переходить',
 'к',
 'описание',
 'тип',
 'смартфон',
 'диагональ',
 'экран',
 'дюйм',
 'емкость',
 'аккумулятор',
 'мач',
 'процессор',
 'helio',
 'g',
 'ядро',
 'ггц',
 'основной',
 'материал',
 'корпус',
 'пластик',
 'о',
 'товар',
 'переходить',
 'к',
 'описание',
 'тип',
 'смартфон',
 'диагональ',
 'экран',
 'дюйм',
 'емкость',
 'аккумулятор',
 'мач',
 'процессор',
 'helio',
 'g',
 'ядро',
 'ггц',
 'основной',
 'материал',
 'корпус',
 'пластик',
 'о',
 'товар',
 'перех

In [2]:
# Получение списка стоп слов
stop_words = []
with open('/home/al/Projects_My/hakaton_ozon_opg/dataset/stop_word_base.txt', 'r') as f:
    for i in f:
        stop_words.append(i.strip())

In [6]:
NAME_START_POINT = '/home/al/Projects_My/hakaton_ozon_opg/dataset/'
list_dir = get_dir(NAME_START_POINT)
#print(list_dir)
categories = list_dir.copy()
array_txt = [] # описание как предложения из лемматизированных слов
m = Mystem()
kk = 0
catY = []
keys_cat = []
all_words_list = []
for sd in list_dir:
    key_one = sd.split('/')[-1].lower()
    print(key_one)
    keys_cat.append(key_one)
    list_dir_two = get_dir(sd)
    for dt in list_dir_two:
        list_text = glob(dt + '/*.txt')
        #print(list_text)
        #print()
        servis_list = []
        # servis_list.append(key_one)
        for nf in list_text:
            dd = get_array_text_with_category(nf, m)
            servis_list += dd
            all_words_list += dd
            #break
        array_txt.append(' '.join(servis_list))
        catY.append(kk)
        # print()
        # print(array_txt[kk])
    kk += 1
    #break

# set_dict = []
# for key in array_txt:
#     print()
#     print(key)
#     print(array_txt[key])
#     dd = dict(array_txt[key])
#     nab = []
#     for kk in dd:
#         if dd[kk] >=2 and kk not in stop_words:
#             nab.append(kk)
#     set_dict.append(nab)
set_all_words = set(all_words_list)
print(keys_cat)
print(len(array_txt))
print(catY)
print(len(set_all_words))

чехлы
наушники
платья
['чехлы', 'наушники', 'платья']
32
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
1109


In [23]:
array_txt[2]

'смартфон накладка камера противоударный силиконовый накладка камера kreiger mix специально спроектировать samsung galaxy a14 4g серьезный периметр обладать усиленный угол смартфон средство воздушный подушка находиться между угол накладка серьезно смягчать удар падение толщина предусматривать камера бортик вокруг камера защищать линза отдельность упругий полиуретан безопасный мягкий ощупь накладка иметься отверстие шнурок помощь смартфон можно носить шея рука силиконовый накладка камера противоударный общий мес samsung galaxy a14 5g samsung galaxy a14 962979473 смартфон фиолетовый код продавец 127716'

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [8]:
X_train, X_test, y_train, y_test = train_test_split(array_txt, catY, test_size=0.20, random_state=42)
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)

In [9]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [12]:
# Метод наивный Байес
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [13]:
X_test_new = vectorizer.transform(X_test)
#print(X_test_new.shape)
X_new_tfidf = tfidf_transformer.transform(X_test_new)
predicted = clf.predict(X_new_tfidf)

In [15]:
kk = 0
for doc, category in zip(y_test, predicted):
    print(doc, category)
    if doc == category: kk +=1
        
print('model ', kk/len(y_test))

2 2
1 1
2 2
1 1
0 0
0 0
2 2
model  1.0


In [8]:
vectorizer.get_feature_names_out()

array(['00', '1031436306', '11', ..., 'юбка', 'являться', 'яркость'],
      dtype=object)

In [9]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 5 ... 0 2 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [48]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=set_all_words)),
                 ('tfid', TfidfTransformer())]).fit(array_txt)
# tf_transformer = TfidfTransformer(use_idf=False).fit(array_txt)
pipe['count']

CountVectorizer(vocabulary={'a', 'aac', 'acs', 'air', 'amoled', 'android',
                            'apparel', 'apple', 'aramid', 'avakyan', 'awesome',
                            'bass', 'best', 'black', 'bluetooth', 'borofone',
                            'boutique', 'broscorp', 'by', 'c', 'camera', 'case',
                            'cavolo', 'chrome', 'club', 'concept', 'ew',
                            'femme', 'fiber', 'g', ...})

In [49]:
pipe.transform(array_txt).shape

(32, 1109)

In [50]:
pipe['tfid'].idf_

array([4.49650756, 3.1102132 , 3.80336038, ..., 4.49650756, 4.49650756,
       4.49650756])

In [41]:
import sklearn.datasets as ds
list_tem = dir(ds)
len(list_tem)
print(list_tem)

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_arff_parser', '_base', '_california_housing', '_covtype', '_kddcup99', '_lfw', '_olivetti_faces', '_openml', '_rcv1', '_samples_generator', '_species_distributions', '_svmlight_format_fast', '_svmlight_format_io', '_twenty_newsgroups', 'clear_data_home', 'descr', 'dump_svmlight_file', 'fetch_20newsgroups', 'fetch_20newsgroups_vectorized', 'fetch_california_housing', 'fetch_covtype', 'fetch_kddcup99', 'fetch_lfw_pairs', 'fetch_lfw_people', 'fetch_olivetti_faces', 'fetch_openml', 'fetch_rcv1', 'fetch_species_distributions', 'get_data_home', 'load_breast_cancer', 'load_diabetes', 'load_digits', 'load_files', 'load_iris', 'load_linnerud', 'load_sample_image', 'load_sample_images', 'load_svmlight_file', 'load_svmlight_files', 'load_wine', 'make_biclusters', 'make_blobs', 'make_checkerboard', 'make_circles', 'make_classification', 'make_friedman1'

In [44]:
from sklearn.datasets import load_iris
from sklearn.datasets import __name__
dd = load_iris()
dd.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [46]:
__name__

'sklearn.datasets'

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
]

# data_train = fetch_20newsgroups(
#     subset="train",
#     categories=categories,
#     shuffle=True,
#     random_state=42,
#     remove=("headers", "footers", "quotes"),
# )
data_all = fetch_20newsgroups()

In [11]:
data_all.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [13]:
data_all.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
data_all.target

array([7, 4, 4, ..., 3, 1, 8])

In [19]:
print(data_all.data[8])

From: holmes7000@iscsvax.uni.edu
Subject: WIn 3.0 ICON HELP PLEASE!
Organization: University of Northern Iowa
Lines: 10

I have win 3.0 and downloaded several icons and BMP's but I can't figure out
how to change the "wallpaper" or use the icons.  Any help would be appreciated.


Thanx,

-Brando

PS Please E-mail me




In [18]:
data_train.keys() #['data'][0]

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [19]:
data_train['target']

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [20]:
data_train['target_names']

['alt.atheism', 'talk.religion.misc']

In [ ]:

data_test = fetch_20newsgroups(
    subset="test",
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=("headers", "footers", "quotes"),
)

print(f"Loading 20 newsgroups dataset for {len(data_train.target_names)} categories:")
print(data_train.target_names)
print(f"{len(data_train.data)} documents")